In [1]:
using DataFrames, CSV, Tables, Plots
using JuMP, Gurobi

In [2]:
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


Gurobi.Env(Ptr{Nothing} @0x000000008169fb90, false, 0)

In [ ]:
baseline = (CSV.File("./baseline.csv") |> Tables.matrix)

In [3]:
predictions = (CSV.File("../Processed data files/XGB_Predictions.csv") |> Tables.matrix);
consumptionUncertainity = (CSV.File("./consumptionUncertainity.csv") |> Tables.matrix)

In [10]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV));
@variable(model, x[1:n,1:T] >= 0, Bin); 
@constraint(model,  sum(x[:,1].*budget) <= B)
for t=2:T
    @constraint(model, [i=1:n], sum(x[:,t-1].*A[i,:]) >= threshold[i]*x[i,t])
    @constraint(model, [i=1:n], sum(x[:,t-1].*A[i,:]) <= threshold[i]+x[i,t])
    @constraint(model, [i=1:n], x[i,t-1] <= x[i,t])
end
@objective(model, Max, sum(x[:,T]))
optimize!(model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 18016 rows, 12010 columns and 94081 nonzeros
Model fingerprint: 0x1ec417ce
Variable types: 0 continuous, 12010 integer (12010 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective -0.0000000
Presolve removed 15886 rows and 10347 columns
Presolve time: 0.46s
Presolved: 2130 rows, 1663 columns, 11092 nonzeros
Variable types: 0 continuous, 1663 integer (1657 binary)
Found heuristic solution: objective 235.0000000

Root relaxation: objective 3.874886e+02, 1448 iterations, 0.09 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  387.48861    0  260  235.00